# Doppler Solve

## Setup

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
import george

In [ ]:
def generate(
    nc=1,
    show=True,
    flux_err=1e-4,
    ydeg=15,
    udeg=2,
    nt=16,
    inc=40,
    veq=60000,
    smoothing=0.075,
    **kwargs
):

    # Instantiate
    wav = np.linspace(642.85, 643.15, 200)
    map = starry.DopplerMap(
        ydeg=ydeg, udeg=udeg, nc=nc, veq=veq, inc=inc, nt=nt, wav=wav
    )

    # Limb darkening (TODO: fix __setitem__)
    if udeg > 0:
        map._u[1] = 0.5
        if udeg > 1:
            map._u[2] = 0.25

    # Component surface images
    if nc == 1:
        images = ["spot"]
    elif nc == 2:
        images = ["star", "spot"]
    else:
        raise NotImplementedError("")

    # Component spectra
    if nc == 1:
        mu = np.array([643.0])
    elif nc == 2:
        mu = np.array([643.025, 642.975])
    else:
        raise NotImplementedError("")
    sig = 0.0085
    dw = map.wav0.reshape(1, -1) - mu.reshape(-1, 1)
    spectra = (1.0 - np.exp(-0.5 * dw ** 2 / sig ** 2))[:nc]

    # Load the component maps
    map.load(maps=images, spectra=spectra, smoothing=smoothing)

    # Show
    if show:
        map.show_components(show_spectra=True)

    # Generate unnormalized data
    flux0 = map.flux(normalize=False)
    flux0 += flux_err * np.random.randn(*flux0.shape)

    # Generate normalized data
    flux = map.flux(normalize=True)
    flux += flux_err * np.random.randn(*flux.shape)

    return map, flux0, flux

## Nonlinear solve (experimental)

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

In [ ]:
map._y[0, 0] = 1
map._y[1:, 0] = 0
map._spectrum = np.ones((map.nc, map.nw0_))
info = map.solve(
    flux,
    solver="nonlinear",
    normalized=True,
    flux_err=settings["flux_err"],
    niter=100000,
)
map.show_components()

In [ ]:
stop

## Solve for everything (experimental)

### One component

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

#### Unnormalized solve

In [ ]:
spectral_cov = george.GP(1e-2 * george.kernels.Matern32Kernel(1e-3)).get_matrix(
    map.wav0
)
map.solve(
    flux0, normalized=False, flux_err=settings["flux_err"], spectral_cov=spectral_cov
)
map.show_components()

#### Normalized solve

In [ ]:
spectral_cov = george.GP(1e-2 * george.kernels.Matern32Kernel(1e-3)).get_matrix(
    map.wav0
)
map.solve(
    flux,
    normalized=True,
    flux_err=settings["flux_err"],
    spectral_cov=spectral_cov,
)
map.show_components()

### Two components

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=2, **settings)

#### Unnormalized solve

In [ ]:
spectral_cov = george.GP(1e-2 * george.kernels.Matern32Kernel(1e-3)).get_matrix(
    map.wav0
)
map.solve(
    flux0,
    normalized=False,
    flux_err=settings["flux_err"],
    spectral_cov=spectral_cov,
    logT0=2,
    nlogT=1000,
)
map.show_components()

#### Normalized solve: TODO

## Solve for the spectrum

### One component

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

#### Unnormalized solve

In [ ]:
map.solve(flux0, normalized=False, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

#### Normalized solve

In [ ]:
map.solve(flux, normalized=True, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

### Two components

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=2, **settings)

#### Unnormalized solve

In [ ]:
map.solve(flux0, normalized=False, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

#### Normalized solve

In [ ]:
map.solve(flux, normalized=True, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

## Solve for the map

### One component

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

In [ ]:
fig, ax = plt.subplots(1, figsize=(3, 6))
ax.plot(map.wav, flux.T + np.linspace(0, 1.5, map.nt).reshape(1, -1), color="k", lw=1)
ax.set_xlabel("wavelength [nm]")
ax.set_ylabel("intensity + offset");

#### Unnormalized solve

In [ ]:
map.solve(flux0, normalized=False, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

#### Normalized solve w/ tempering

In [ ]:
map.solve(flux, normalized=True, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

### Two components

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=2, **settings)

In [ ]:
fig, ax = plt.subplots(1, figsize=(3, 6))
ax.plot(map.wav, flux.T + np.linspace(0, 1.5, map.nt).reshape(1, -1), color="k", lw=1)
ax.set_xlabel("wavelength [nm]")
ax.set_ylabel("intensity + offset");

#### Unnormalized solve

In [ ]:
map.solve(flux0, normalized=False, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

#### Normalized solve w/ tempering

In [ ]:
map.solve(flux, normalized=True, fix_spectrum=True, flux_err=settings["flux_err"])
map.show_components(show_spectra=False)

### One component

#### Generate the data

In [ ]:
settings = dict(flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075)
map, flux0, flux = generate(nc=1, **settings)

## Solve for everything